# Demo de algoritmos para almacenar passwords

In [ ]:
password = b'cbda'

In [ ]:
ALPHABET = 'abcd'

## Probando las funciones hash clásicas: MD5, SHA1 y SHA256

In [ ]:
import hashlib

In [ ]:
md5_digest = hashlib.md5(password).hexdigest()
print(md5_digest)

In [ ]:
sha1_digest = hashlib.sha1(password).hexdigest()
print(sha1_digest)

In [ ]:
sha256_digest = hashlib.sha256(password).hexdigest()
print(sha256_digest)

Ahora, una función que me permita generar todas las passwords posibles (del largo del alfabeto) para poder hacer fuerza bruta, y una segunda que me permita hacer la fuerza bruta para cada algoritmo:

In [ ]:
import itertools

def get_bruteforce_string_list():
    prod = itertools.product(ALPHABET, repeat = len(ALPHABET))
    return (("".join(map(str, a)).encode('utf-8') for a in prod))
def bruteforce_hash(hash_object, password_hash):
    for attempt in get_bruteforce_string_list():
        h = hashlib.new(hash_object)
        h.update(attempt)
        if h.hexdigest() == password_hash:
            print('PASSWORD FOUND: %s' % attempt)
            break
def bruteforce_plaintext():
    for attempt in get_bruteforce_string_list():
        if attempt == password:
            print('PASSWORD FOUND: %s' % attempt)
            break

#### Fuerza bruta para texto plano

In [ ]:
timeit_plaintext = %timeit -n1 -r1 -q -o bruteforce_plaintext()

#### Fuerza bruta para MD5

In [ ]:
timeit_md5 = %timeit -n1 -r1 -q -o bruteforce_hash('md5', md5_digest)

#### Fuerza bruta para SHA1

In [ ]:
timeit_sha1 = %timeit -n1 -r1 -q -o bruteforce_hash('sha1', sha1_digest)

#### Fuerza bruta para SHA256

In [ ]:
timeit_sha256 = %timeit -n1 -r1 -q -o bruteforce_hash('sha256', sha256_digest)

## Ahora con una función KDF: BCrypt

In [ ]:
import bcrypt

Generación de una sal. A la función `gensalt` se le puede dar un parámetro de costo. Por defecto es 12. Ver <https://en.wikipedia.org/wiki/Bcrypt>

In [ ]:
salt = bcrypt.gensalt()
print(salt)

Generación del KDF o "digest" respectivo:

In [ ]:
kdf_bcrypt = bcrypt.hashpw(password, salt)
print(kdf_bcrypt)

#### Fuerza bruta para BCrypt

In [ ]:
def bruteforce_bcrypt():
    for attempt in get_bruteforce_string_list():
        if bcrypt.hashpw(attempt, kdf_bcrypt) == kdf_bcrypt:
            print('PASSWORD FOUND: %s' % attempt)
            break
timeit_bcrypt = %timeit -n1 -r1 -q -o bruteforce_bcrypt()

## Resultados

In [ ]:
import pandas as pd

data = {
    'Tipo': ['Texto plano', 'MD5', 'SHA1', 'SHA256', 'KDF (BCrypt)'],
    'Tiempo (seg.)': [timeit_plaintext.best, timeit_md5.best, timeit_sha1.best, timeit_sha256.best, timeit_bcrypt.best]
}

df = pd.DataFrame(data, columns = ['Tipo', 'Tiempo (seg.)'])
df = df.set_index('Tipo')

print(df)